# Real Time Code

## Libraries

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

## Code

In [ ]:

def run_yolo(model_path, mode, confidence_threshold= 0.5, device=0):
    """
    Runs a YOLOv8 detection or segmentation loop on the webcam.

    Args:
        model_path (str): Path to the .pt model (e.g. 'yolov8n-seg.pt' or 'yolov8n.pt').
        mode (str): 'detect' for bounding boxes, 'segment' for segmentation masks + boxes.
        confidence_threshold: Confidence threshold between 0.0 and 1.0.
        device: Camera device index (default 0).
    """
    # Load the YOLO model
    model = YOLO(model_path)

    # Open the webcam
    cap = cv2.VideoCapture(device)
    if not cap.isOpened():
        raise RuntimeError(f"Could not open camera index {device}")

    # Prepare random colors for segmentation
    if mode == "segment":
        num_classes = len(model.names)
        colors = np.random.randint(0, 255, size=(num_classes, 3), dtype=np.uint8)

    window_name = f"YOLO - {mode.capitalize()} (for exit press 'Q')"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Perform inference
            results = model(frame)
            result = results[0]

            if mode == "detect":
                display_frame = frame.copy()
                # Draw bounding boxes
                for box in result.boxes:
                    conf = float(box.conf)
                    if conf < confidence_threshold:
                        continue
                    x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                    cls = int(box.cls.cpu().numpy()[0])
                    label = f"{model.names[cls]}: {conf:.2f}"
                    cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(display_frame, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            elif mode == "segment":
                # Check if masks are available
                if result.masks is None:
                    raise RuntimeError(
                        f"Model '{model_path}' does not support segmentation. "
                        "Please use a segmentation model like 'yolov8n-seg.pt'."
                    )
                display_frame = frame.copy()

                # Extract data
                boxes = result.boxes.xyxy.cpu().numpy()
                scores = result.boxes.conf.cpu().numpy()
                classes = result.boxes.cls.cpu().numpy().astype(int)
                masks = result.masks.data.cpu().numpy()

                # Overlay masks and draw boxes
                overlay = frame.copy()
                for i, conf in enumerate(scores):
                    if conf < confidence_threshold:
                        continue
                    x1, y1, x2, y2 = map(int, boxes[i])
                    cls = classes[i]
                    mask = masks[i] > 0.5

                    color = [int(c) for c in colors[cls]]
                    mask_color = np.zeros_like(frame, dtype=np.uint8)
                    mask_color[mask] = color

                    # Blend mask onto the frame
                    overlay = cv2.addWeighted(overlay, 1.0, mask_color, 0.5, 0)
                    cv2.rectangle(overlay, (x1, y1), (x2, y2), color, 2)
                    label = f"{model.names[cls]}: {conf:.2f}"
                    cv2.putText(overlay, label, (x1, y1 - 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
                display_frame = overlay
            else:
                raise ValueError("mode must be 'detect' or 'segment'")

            # Show the frame
            cv2.imshow(window_name, display_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    finally:
        cap.release()
        cv2.destroyAllWindows()


#----------------------------------------------------------------
#modes=["detect", "segment"]
#device=0 (webcam)

run_yolo("yolov8n-seg.pt", "segment", confidence_threshold=0.5,device=0)
